# 1. Introduction

*   What is the impact of each marketing strategy and sales visit on Sales (Amount Collected)?
*   Is the same strategy valid for all the different client types?



# 2. Data loading and quality checks

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats

In [2]:
data=pd.read_csv('/content/sample_data/Campaign-Data.csv')
data.head()

,Client ID,Client Type,Number of Customers,Montly Target,Zip Code,Calendardate,Amount Collected,Unit Sold,Campaign (Email),Campaign (Flyer),Campaign (Phone),Sales Contact 1,Sales Contact 2,Sales Contact 3,Sales Contact 4,Sales Contact 5,Number of Competition
0,ID-987275,Medium Facility,2800,125,1003,16-01-2014,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Low
1,ID-987275,Medium Facility,2800,125,1003,16-02-2014,3409460,24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,322500.0,Low
2,ID-987275,Medium Facility,2800,125,1003,18-03-2014,10228384,75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Low
3,ID-987275,Medium Facility,2800,125,1003,18-04-2014,17047304,123,0.0,0.0,0.0,0.0,3547500.0,1290000.0,0.0,0.0,Low
4,ID-987275,Medium Facility,2800,125,1003,19-05-2014,23866224,171,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Low


In [3]:
data.columns

Index(['Client ID', 'Client Type', 'Number of Customers', 'Montly Target',
       'Zip Code', 'Calendardate', 'Amount Collected', 'Unit Sold',
       'Campaign (Email)', 'Campaign (Flyer)', 'Campaign (Phone)',
       'Sales Contact 1', 'Sales Contact 2', 'Sales Contact 3',
       'Sales Contact 4', 'Sales Contact 5', 'Number of Competition'],
      dtype='object')

In [4]:
data['Client Type'].value_counts()

,count
Client Type,
Large Facility,1368
Small Facility,840
Medium Facility,504
Private Facility,264


# 3. Feature additions and engineering

In [5]:
# creating additional features
data['Calendardate'] = pd.to_datetime(data['Calendardate'],dayfirst=True)
data['Year'] = data['Calendardate'].dt.year
data['Month'] = data['Calendardate'].dt.month

In [6]:
data.head()

,Client ID,Client Type,Number of Customers,Montly Target,Zip Code,Calendardate,Amount Collected,Unit Sold,Campaign (Email),Campaign (Flyer),Campaign (Phone),Sales Contact 1,Sales Contact 2,Sales Contact 3,Sales Contact 4,Sales Contact 5,Number of Competition,Year,Month
0,ID-987275,Medium Facility,2800,125,1003,2014-01-16,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Low,2014,1
1,ID-987275,Medium Facility,2800,125,1003,2014-02-16,3409460,24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,322500.0,Low,2014,2
2,ID-987275,Medium Facility,2800,125,1003,2014-03-18,10228384,75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Low,2014,3
3,ID-987275,Medium Facility,2800,125,1003,2014-04-18,17047304,123,0.0,0.0,0.0,0.0,3547500.0,1290000.0,0.0,0.0,Low,2014,4
4,ID-987275,Medium Facility,2800,125,1003,2014-05-19,23866224,171,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Low,2014,5


# 4. EDA
**4.1 Exploring and Understanding basics of data**<br>


1.   Distribution of data across different accounts
2.   Difference of sales in account types (using categorical mean)


**4.2 Statistical Analysis - Answering questions**



1.   Impact of Marketing Startegy on sales (using correlation, regression and decision trees)
2.   Impact of competition on sales
3. How different types of client can have different strategies (broken down Q1 and Q2 based on Account Type)



In [7]:
data['Client Type'].value_counts(normalize=True)

,proportion
Client Type,
Large Facility,0.459677
Small Facility,0.282258
Medium Facility,0.169355
Private Facility,0.088710


In [8]:
pd.crosstab(data['Number of Competition'],data['Client Type'],margins=True,normalize='columns')

Client Type,Large Facility,Medium Facility,Private Facility,Small Facility,All
Number of Competition,,,,,
High,0.166667,0.166667,0.166667,0.166667,0.166667
Low,0.833333,0.833333,0.833333,0.833333,0.833333


In [9]:
data2=data.drop(['Client ID','Client Type'],axis=1)
data2

,Number of Customers,Montly Target,Zip Code,Calendardate,Amount Collected,Unit Sold,Campaign (Email),Campaign (Flyer),Campaign (Phone),Sales Contact 1,Sales Contact 2,Sales Contact 3,Sales Contact 4,Sales Contact 5,Number of Competition,Year,Month
0,2800,125,1003,2014-01-16,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Low,2014,1
1,2800,125,1003,2014-02-16,3409460,24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,322500.0,Low,2014,2
2,2800,125,1003,2014-03-18,10228384,75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Low,2014,3
3,2800,125,1003,2014-04-18,17047304,123,0.0,0.0,0.0,0.0,3547500.0,1290000.0,0.0,0.0,Low,2014,4
4,2800,125,1003,2014-05-19,23866224,171,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Low,2014,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2971,800,20,1003,2015-08-16,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Low,2015,8
2972,800,20,1003,2015-09-16,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,High,2015,9
2973,800,20,1003,2015-10-16,0,0,0.0,0.0,0.0,0.0,0.0,2580000.0,0.0,0.0,High,2015,10
2974,800,20,1003,2015-11-16,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,High,2015,11


In [10]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2976 entries, 0 to 2975
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Number of Customers    2976 non-null   int64         
 1   Montly Target          2976 non-null   int64         
 2   Zip Code               2976 non-null   int64         
 3   Calendardate           2976 non-null   datetime64[ns]
 4   Amount Collected       2976 non-null   int64         
 5   Unit Sold              2976 non-null   int64         
 6   Campaign (Email)       2976 non-null   float64       
 7   Campaign (Flyer)       2976 non-null   float64       
 8   Campaign (Phone)       2976 non-null   float64       
 9   Sales Contact 1        2976 non-null   float64       
 10  Sales Contact 2        2976 non-null   float64       
 11  Sales Contact 3        2976 non-null   float64       
 12  Sales Contact 4        2976 non-null   float64       
 13  Sal

In [11]:
data2.groupby('Number of Competition').mean()

,Number of Customers,Montly Target,Zip Code,Calendardate,Amount Collected,Unit Sold,Campaign (Email),Campaign (Flyer),Campaign (Phone),Sales Contact 1,Sales Contact 2,Sales Contact 3,Sales Contact 4,Sales Contact 5,Year,Month
Number of Competition,,,,,,,,,,,,,,,,
High,1456.935484,75.080645,1003.0,2015-10-31 18:00:00,2.974789e+07,213.127016,105398.938508,994046.717540,45198.036895,146945.564516,2.685333e+06,1.786754e+06,72172.379032,8452.620968,2015.0,10.5
Low,1456.935484,75.080645,1003.0,2014-10-31 22:48:00,1.445570e+07,103.132258,150862.165766,623692.979839,26693.304194,128219.758065,1.890916e+06,1.883634e+06,70481.854839,15864.919355,2014.4,5.7


Amount collected for High are double than Low. Units sold are also almost double for High than Low

In [12]:
data2=data.drop(['Number of Competition','Client ID'],axis=1)

In [13]:
data2.groupby('Client Type').mean()

,Number of Customers,Montly Target,Zip Code,Calendardate,Amount Collected,Unit Sold,Campaign (Email),Campaign (Flyer),Campaign (Phone),Sales Contact 1,Sales Contact 2,Sales Contact 3,Sales Contact 4,Sales Contact 5,Year,Month
Client Type,,,,,,,,,,,,,,,,
Large Facility,1380.842105,71.578947,1003.0,2014-12-31 18:00:00.000000000,1.999880e+07,143.098684,142273.609649,8.192056e+05,45595.436623,133667.763158,2.034013e+06,2.017039e+06,119287.280702,16266.447368,2014.5,6.5
Medium Facility,3940.761905,202.857143,1003.0,2014-12-31 18:00:00.000000000,4.075997e+07,290.583333,437217.097817,1.552603e+06,49176.847619,398645.833333,4.822783e+06,4.698646e+06,85104.166667,33273.809524,2014.5,6.5
Private Facility,400.727273,20.454545,1003.0,2014-12-31 18:00:00.000000000,5.030246e+06,35.784091,5183.715152,2.272919e+05,5522.470455,1221.590909,6.376705e+05,4.434375e+05,3664.772727,12215.909091,2014.5,6.5
Small Facility,422.514286,21.285714,1003.0,2014-12-31 17:59:59.999999744,1.637759e+06,11.689286,11975.986310,9.120875e+04,0.000000,8062.500000,7.617143e+05,3.727946e+05,4223.214286,1535.714286,2014.5,6.5


In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2976 entries, 0 to 2975
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Client ID              2976 non-null   object        
 1   Client Type            2976 non-null   object        
 2   Number of Customers    2976 non-null   int64         
 3   Montly Target          2976 non-null   int64         
 4   Zip Code               2976 non-null   int64         
 5   Calendardate           2976 non-null   datetime64[ns]
 6   Amount Collected       2976 non-null   int64         
 7   Unit Sold              2976 non-null   int64         
 8   Campaign (Email)       2976 non-null   float64       
 9   Campaign (Flyer)       2976 non-null   float64       
 10  Campaign (Phone)       2976 non-null   float64       
 11  Sales Contact 1        2976 non-null   float64       
 12  Sales Contact 2        2976 non-null   float64       
 13  Sal

In [15]:
df=data.drop(['Client ID','Zip Code','Calendardate'],axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2976 entries, 0 to 2975
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Client Type            2976 non-null   object 
 1   Number of Customers    2976 non-null   int64  
 2   Montly Target          2976 non-null   int64  
 3   Amount Collected       2976 non-null   int64  
 4   Unit Sold              2976 non-null   int64  
 5   Campaign (Email)       2976 non-null   float64
 6   Campaign (Flyer)       2976 non-null   float64
 7   Campaign (Phone)       2976 non-null   float64
 8   Sales Contact 1        2976 non-null   float64
 9   Sales Contact 2        2976 non-null   float64
 10  Sales Contact 3        2976 non-null   float64
 11  Sales Contact 4        2976 non-null   float64
 12  Sales Contact 5        2976 non-null   float64
 13  Number of Competition  2976 non-null   object 
 14  Year                   2976 non-null   int32  
 15  Mont

In [16]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(df['Client Type'])
df['Client Type'] = le.transform(df['Client Type'])

le1 = LabelEncoder()
le1.fit(df['Number of Competition'])
df['Number of Competition'] = le1.transform(df['Number of Competition'])

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2976 entries, 0 to 2975
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Client Type            2976 non-null   int64  
 1   Number of Customers    2976 non-null   int64  
 2   Montly Target          2976 non-null   int64  
 3   Amount Collected       2976 non-null   int64  
 4   Unit Sold              2976 non-null   int64  
 5   Campaign (Email)       2976 non-null   float64
 6   Campaign (Flyer)       2976 non-null   float64
 7   Campaign (Phone)       2976 non-null   float64
 8   Sales Contact 1        2976 non-null   float64
 9   Sales Contact 2        2976 non-null   float64
 10  Sales Contact 3        2976 non-null   float64
 11  Sales Contact 4        2976 non-null   float64
 12  Sales Contact 5        2976 non-null   float64
 13  Number of Competition  2976 non-null   int64  
 14  Year                   2976 non-null   int32  
 15  Mont

In [17]:
df.head()

,Client Type,Number of Customers,Montly Target,Amount Collected,Unit Sold,Campaign (Email),Campaign (Flyer),Campaign (Phone),Sales Contact 1,Sales Contact 2,Sales Contact 3,Sales Contact 4,Sales Contact 5,Number of Competition,Year,Month
0,1,2800,125,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,2014,1
1,1,2800,125,3409460,24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,322500.0,1,2014,2
2,1,2800,125,10228384,75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,2014,3
3,1,2800,125,17047304,123,0.0,0.0,0.0,0.0,3547500.0,1290000.0,0.0,0.0,1,2014,4
4,1,2800,125,23866224,171,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,2014,5


In [18]:
# Map for encoding values of Client Type
print(df['Client Type'].unique())
le.inverse_transform(df['Client Type'].unique())

[1 0 3 2]


array(['Medium Facility', 'Large Facility ', 'Small Facility ',
       'Private Facility'], dtype=object)

In [19]:
print(df['Number of Competition'].unique())
le1.inverse_transform(df['Number of Competition'].unique())

[1 0]


array(['Low', 'High'], dtype=object)

In [20]:
df.corr()['Amount Collected']

,Amount Collected
Client Type,-0.286855
Number of Customers,0.607496
Montly Target,0.608204
Amount Collected,1.000000
Unit Sold,0.997515
Campaign (Email),0.248235
Campaign (Flyer),0.444337
Campaign (Phone),0.034858
Sales Contact 1,0.277478
Sales Contact 2,0.552112


## Correlation Analysis

In [21]:
import seaborn as sns

In [22]:
cm=sns.light_palette("green",as_cmap=True)
correlation_analysis=pd.DataFrame(df[['Amount Collected','Campaign (Email)',
                      'Campaign (Flyer)','Campaign (Phone)','Sales Contact 1',
                      'Sales Contact 2','Sales Contact 3','Sales Contact 4',
                      'Sales Contact 5']].corr()['Amount Collected']).reset_index()
correlation_analysis.columns=['Impacting Variable','Degree of Linear Impact (Correlation)']
correlation_analysis=correlation_analysis[correlation_analysis['Impacting Variable']!='Amount Collected']
correlation_analysis=correlation_analysis.sort_values('Degree of Linear Impact (Correlation)',ascending=False)
correlation_analysis.style.background_gradient(cmap=cm)


,Impacting Variable,Degree of Linear Impact (Correlation)
5,Sales Contact 2,0.552112
2,Campaign (Flyer),0.444337
6,Sales Contact 3,0.357887
4,Sales Contact 1,0.277478
1,Campaign (Email),0.248235
7,Sales Contact 4,0.236165
8,Sales Contact 5,0.095795
3,Campaign (Phone),0.034858


In [23]:
cm=sns.light_palette("green",as_cmap=True)
correlation_analysis=pd.DataFrame(data2.groupby('Client Type')[['Amount Collected','Campaign (Email)',
                      'Campaign (Flyer)','Campaign (Phone)','Sales Contact 1',
                      'Sales Contact 2','Sales Contact 3','Sales Contact 4',
                      'Sales Contact 5']].corr()['Amount Collected']).reset_index()
correlation_analysis=correlation_analysis.sort_values(['Client Type','Amount Collected'],ascending=False)

correlation_analysis.columns=['Acc. Type','Variable Impact on Sales','Impact']
correlation_analysis=correlation_analysis[correlation_analysis['Variable Impact on Sales']!='Amount Collected'].reset_index(drop=True)

correlation_analysis.style.background_gradient(cmap=cm)


,Acc. Type,Variable Impact on Sales,Impact
0,Small Facility,Sales Contact 2,0.219759
1,Small Facility,Sales Contact 3,0.068269
2,Small Facility,Campaign (Email),0.060200
3,Small Facility,Campaign (Flyer),0.040600
4,Small Facility,Sales Contact 4,0.024401
5,Small Facility,Sales Contact 5,0.000927
6,Small Facility,Sales Contact 1,-0.015594
7,Small Facility,Campaign (Phone),nan
8,Private Facility,Sales Contact 2,0.574481
9,Private Facility,Campaign (Flyer),0.280098


## Regression Analysis (Market Sales and Strategies)

In [24]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
data.columns=[string.replace(" ",'_') for string in data.columns]
data.columns=[string.replace("(",'') for string in data.columns]
data.columns=[string.replace(")","") for string in data.columns]
results = smf.ols('Amount_Collected ~ Campaign_Email+Campaign_Flyer+Campaign_Phone+Sales_Contact_1+Sales_Contact_2+Sales_Contact_3+Sales_Contact_4+Sales_Contact_5', data=data).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:       Amount_Collected   R-squared:                       0.480
Model:                            OLS   Adj. R-squared:                  0.478
Method:                 Least Squares   F-statistic:                     342.1
Date:                Fri, 16 Aug 2024   Prob (F-statistic):               0.00
Time:                        08:34:08   Log-Likelihood:                -54512.
No. Observations:                2976   AIC:                         1.090e+05
Df Residuals:                    2967   BIC:                         1.091e+05
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept        1.481e+06   5.12e+05     

In [25]:
df2 = pd.read_html(results.summary().tables[1].as_html(),header=0,index_col=0)[0]

<ipython-input-25-9a31dfa70dff>:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df2 = pd.read_html(results.summary().tables[1].as_html(),header=0,index_col=0)[0]


In [26]:
df2 = df2.reset_index()
df2 = df2[df2['P>|t|']<0.05][['index','coef']]
df2

,index,coef
0,Intercept,1.481000e+06
2,Campaign_Flyer,3.337600e+00
4,Sales_Contact_1,4.236800e+00
5,Sales_Contact_2,3.638200e+00
6,Sales_Contact_3,2.343200e+00
7,Sales_Contact_4,1.094780e+01


In [37]:
consolidated_summary=pd.DataFrame()
for acctype in list(set(list(data['Client_Type']))):
  temp = data[data['Client_Type']==acctype].copy()
  results= smf.ols('Amount_Collected ~ Campaign_Email+Campaign_Flyer+Campaign_Phone+Sales_Contact_1+Sales_Contact_2+Sales_Contact_3+Sales_Contact_4+Sales_Contact_5', data=temp).fit()
  df2 = pd.read_html(results.summary().tables[1].as_html(),header=0,index_col=0)[0].reset_index()
  df2 = df2[df2['P>|t|']<0.05][['index','coef']]
  df2.columns = ['Variable','Coefficent (Impact)']
  df2['Account Type'] = acctype
  df2 = df2.sort_values('Coefficent (Impact)',ascending=False)
  df2 = df2[df2['Variable']!='Intercept']
  print(acctype)
  consolidated_summary = pd.concat([consolidated_summary, df2], ignore_index=True)
  print(df2)

Small Facility 
          Variable  Coefficent (Impact)     Account Type
5  Sales_Contact_2             0.810100  Small Facility 
3   Campaign_Phone            -0.000003  Small Facility 
Private Facility
          Variable  Coefficent (Impact)      Account Type
5  Sales_Contact_2               6.6223  Private Facility
Medium Facility
          Variable  Coefficent (Impact)     Account Type
2   Campaign_Flyer               4.1059  Medium Facility
5  Sales_Contact_2               3.5778  Medium Facility
4  Sales_Contact_1               3.1365  Medium Facility
6  Sales_Contact_3               2.1174  Medium Facility
Large Facility 
          Variable  Coefficent (Impact)     Account Type
4  Sales_Contact_1              11.6731  Large Facility 
7  Sales_Contact_4              10.6145  Large Facility 
5  Sales_Contact_2               4.0031  Large Facility 
2   Campaign_Flyer               2.7204  Large Facility 
6  Sales_Contact_3               2.0316  Large Facility 
3   Campaign_Phone   

<ipython-input-37-374825807dbc>:5: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df2 = pd.read_html(results.summary().tables[1].as_html(),header=0,index_col=0)[0].reset_index()
<ipython-input-37-374825807dbc>:5: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df2 = pd.read_html(results.summary().tables[1].as_html(),header=0,index_col=0)[0].reset_index()
<ipython-input-37-374825807dbc>:5: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df2 = pd.read_html(results.summary().tables[1].as_html(),header=0,index_col=0)[0].reset_index()
<ipython-input-37-374825807dbc>:5: FutureWarning: Passing literal html to 'read_html' is depre

In [43]:
consolidated_summary=pd.DataFrame()
for acctype in list(set(list(data['Client_Type']))):
  temp = data[data['Client_Type']==acctype].copy()
  results= smf.ols('Amount_Collected ~ Campaign_Email+Campaign_Flyer+Campaign_Phone+Sales_Contact_1+Sales_Contact_2+Sales_Contact_3+Sales_Contact_4+Sales_Contact_5', data=temp).fit()
  df2 = pd.read_html(results.summary().tables[1].as_html(),header=0,index_col=0)[0].reset_index()
  df2 = df2[df2['P>|t|']<0.05][['index','coef']]
  df2.columns = ['Variable','Coefficent (Impact)']
  df2['Account Type'] = acctype
  df2 = df2.sort_values('Coefficent (Impact)',ascending=False)
  df2 = df2[df2['Variable']!='Intercept']
  print( "\033[1m "+str(acctype)+"  \033[0m")
  consolidated_summary = pd.concat([consolidated_summary, df2], ignore_index=True)
  print(results.summary())

 Small Facility   
                            OLS Regression Results                            
Dep. Variable:       Amount_Collected   R-squared:                       0.054
Model:                            OLS   Adj. R-squared:                  0.046
Method:                 Least Squares   F-statistic:                     6.829
Date:                Fri, 16 Aug 2024   Prob (F-statistic):           6.63e-08
Time:                        09:30:03   Log-Likelihood:                -14125.
No. Observations:                 840   AIC:                         2.827e+04
Df Residuals:                     832   BIC:                         2.830e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept        8.789e

<ipython-input-43-3e51d08add7f>:5: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df2 = pd.read_html(results.summary().tables[1].as_html(),header=0,index_col=0)[0].reset_index()
<ipython-input-43-3e51d08add7f>:5: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df2 = pd.read_html(results.summary().tables[1].as_html(),header=0,index_col=0)[0].reset_index()
<ipython-input-43-3e51d08add7f>:5: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df2 = pd.read_html(results.summary().tables[1].as_html(),header=0,index_col=0)[0].reset_index()
<ipython-input-43-3e51d08add7f>:5: FutureWarning: Passing literal html to 'read_html' is depre

# 5. Recommendations


In [44]:
consolidated_summary

,Variable,Coefficent (Impact),Account Type
0,Sales_Contact_2,0.810100,Small Facility
1,Campaign_Phone,-0.000003,Small Facility
2,Sales_Contact_2,6.622300,Private Facility
3,Campaign_Flyer,4.105900,Medium Facility
4,Sales_Contact_2,3.577800,Medium Facility
5,Sales_Contact_1,3.136500,Medium Facility
6,Sales_Contact_3,2.117400,Medium Facility
7,Sales_Contact_1,11.673100,Large Facility
8,Sales_Contact_4,10.614500,Large Facility
9,Sales_Contact_2,4.003100,Large Facility


In [47]:
consolidated_summary.columns = ['Variable','ROI','Account Type']
consolidated_summary['ROI'] = consolidated_summary['ROI'].apply(lambda x: round(x,1))
consolidated_summary.style.background_gradient(cmap='RdYlGn')

,Variable,ROI,Account Type
0,Sales_Contact_2,0.800000,Small Facility
1,Campaign_Phone,-0.000000,Small Facility
2,Sales_Contact_2,6.600000,Private Facility
3,Campaign_Flyer,4.100000,Medium Facility
4,Sales_Contact_2,3.600000,Medium Facility
5,Sales_Contact_1,3.100000,Medium Facility
6,Sales_Contact_3,2.100000,Medium Facility
7,Sales_Contact_1,11.700000,Large Facility
8,Sales_Contact_4,10.600000,Large Facility
9,Sales_Contact_2,4.000000,Large Facility


In [48]:
def format (x):
  return "${:.1f}".format(x)
consolidated_summary['ROI']=consolidated_summary['ROI'].apply(format)

In [49]:
consolidated_summary.columns = ['Variable','ROI','Account Type']
consolidated_summary.style.background_gradient(cmap='RdYlGn')

,Variable,ROI,Account Type
0,Sales_Contact_2,$0.8,Small Facility
1,Campaign_Phone,$-0.0,Small Facility
2,Sales_Contact_2,$6.6,Private Facility
3,Campaign_Flyer,$4.1,Medium Facility
4,Sales_Contact_2,$3.6,Medium Facility
5,Sales_Contact_1,$3.1,Medium Facility
6,Sales_Contact_3,$2.1,Medium Facility
7,Sales_Contact_1,$11.7,Large Facility
8,Sales_Contact_4,$10.6,Large Facility
9,Sales_Contact_2,$4.0,Large Facility
